# Análisis de Datos de Google Sheets

Este notebook permite cargar, visualizar y analizar datos nutricionales de jugadores de rugby desde Google Sheets.

In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image
import os

# Configurar estilo de visualización
plt.style.use('ggplot')
sns.set_style('whitegrid')
%matplotlib inline

## Carga de Datos desde Google Sheets

La siguiente función permite cargar datos directamente desde una hoja de Google Sheets compartida.

In [ ]:
def cargar_datos_google_sheets(sheet_url=None):
    if sheet_url is None:
        # URL de la hoja de cálculo compartida por defecto
        sheet_url = "https://docs.google.com/spreadsheets/d/17rtbwMwGMXvoE4sXdGNXXr19z73mdCSjxTApoG5v_6o/edit?gid=0#gid=0"
    
    # Extraer el ID de la hoja de cálculo de la URL
    try:
        if 'spreadsheets/d/' in sheet_url:
            sheet_id = sheet_url.split('spreadsheets/d/')[1].split('/')[0]
        else:
            # Si no tiene el formato esperado
            raise ValueError("La URL no parece ser una URL válida de Google Sheets")
        
        # Construir la URL para exportar como CSV
        csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
        
        # Leer los datos directamente desde la URL
        print(f"Intentando cargar datos desde: {csv_url}")
        df = pd.read_csv(csv_url)
        print(f"✅ Datos cargados exitosamente: {df.shape[0]} filas y {df.shape[1]} columnas")
        
        # Convertir fechas si hay una columna de fecha
        if 'fecha' in df.columns:
            # Lista de formatos a probar para mayor flexibilidad
            formatos = ['%d-%m-%Y', '%d/%m/%Y', '%Y-%m-%d', '%d-%m-%y', '%d/%m/%y']
            
            # Intentar cada formato hasta que uno funcione
            fecha_convertida = False
            for formato in formatos:
                try:
                    # Intentar convertir con el formato actual
                    df['fecha'] = pd.to_datetime(df['fecha'], format=formato, errors='coerce')
                    
                    # Si no hay fechas NaN, hemos encontrado el formato correcto
                    if not df['fecha'].isna().any():
                        fecha_convertida = True
                        print(f"✅ Fechas convertidas correctamente usando el formato: {formato}")
                        break
                except:
                    continue
            
            # Si aún hay fechas nulas después de intentar todos los formatos
            if df['fecha'].isna().any():
                # Detectar filas con problemas
                filas_problematicas = df[df['fecha'].isna()]
                print(f"⚠️ Algunas fechas no pudieron ser convertidas. Verificar que el formato sea día-mes-año (ej: 3-2-2025)")
                print(f"⚠️ Ejemplo de valor problemático: {filas_problematicas['fecha'].iloc[0] if len(filas_problematicas) > 0 else 'No disponible'}")
            
            # Crear columna con formato legible
            df['fecha_str'] = df['fecha'].dt.strftime('%d/%m/%Y')
        else:
            print("⚠️ No se encontró la columna 'fecha' en los datos. Verificar estructura del Google Sheet.")
        
        return df
    except Exception as e:
        print(f"❌ Error al cargar datos: {e}")
        # Proporcionar datos de ejemplo en caso de error
        print("Generando datos de ejemplo para demostración...")
        fechas = pd.date_range(start='2024-01-01', periods=6, freq='MS')
        datos = {
            'fecha': fechas,
            'Peso (kg)': [95.2, 94.8, 93.5, 92.9, 92.3, 91.8],
            'altura': [1.88] * 6,
            'masa_muscular': [42.5, 42.8, 43.2, 43.5, 43.7, 43.9],
            'masa_adiposa': [24.3, 23.5, 22.1, 21.2, 20.5, 19.8],
            'masa_osea': [15.1, 15.1, 15.1, 15.2, 15.2, 15.2],
            'pliegue_triceps': [15.2, 14.8, 14.1, 13.7, 13.2, 12.8],
            'pliegue_subescapular': [16.8, 16.3, 15.7, 15.2, 14.8, 14.3],
            'pliegue_suprailiaco': [18.2, 17.5, 16.8, 16.2, 15.6, 15.1],
            'pliegue_abdominal': [22.5, 21.8, 20.5, 19.8, 19.2, 18.5],
            'pliegue_muslo': [15.7, 15.2, 14.6, 14.2, 13.8, 13.5],
            'pliegue_pantorrilla': [10.2, 9.8, 9.5, 9.2, 8.9, 8.7],
        }
        df = pd.DataFrame(datos)
        df['fecha_str'] = df['fecha'].dt.strftime('%d/%m/%Y')
        return df


In [5]:
def normalizar_valores_nan(dataframe):
    """
    Normaliza el DataFrame reemplazando los valores NaN por 0.
    
    Args:
        dataframe (pd.DataFrame): El DataFrame a normalizar
        
    Returns:
        pd.DataFrame: DataFrame con valores NaN reemplazados por 0
    """
    try:
        # Crear una copia para no modificar el original directamente
        df_normalizado = dataframe.copy()
        
        # Reemplazar todos los valores NaN por 0
        df_normalizado = df_normalizado.fillna(0)
        
        # Contar cuántos valores fueron reemplazados
        total_nan = dataframe.isna().sum().sum()
        print(f"✅ Se han normalizado {total_nan} valores NaN a 0 en el DataFrame")
        
        # Mostrar un resumen de las columnas afectadas
        columnas_afectadas = dataframe.columns[dataframe.isna().any()].tolist()
        if columnas_afectadas:
            print("\nColumnas normalizadas:")
            for col in columnas_afectadas:
                num_nan = dataframe[col].isna().sum()
                print(f"- {col}: {num_nan} valores reemplazados")
                
        return df_normalizado
    except Exception as e:
        print(f"❌ Error al normalizar valores: {e}")
        return dataframe  # Devolver el DataFrame original en caso de error


## Carga de Datos

Puedes usar la URL por defecto o introducir tu propia URL de Google Sheets.

In [6]:
# Puedes colocar tu URL de Google Sheets aquí si deseas usar otra diferente
# Si dejas este campo en blanco, se usará la URL predeterminada
mi_url = ""  # Ejemplo: "https://docs.google.com/spreadsheets/d/tu_id_sheet/edit?usp=sharing"

# Cargar los datos
if mi_url:
    df = cargar_datos_google_sheets(mi_url)
else:
    df = cargar_datos_google_sheets()

# Normalizar los valores NaN a 0
df_normalizado = normalizar_valores_nan(df)

# Usar el DataFrame normalizado para el resto del análisis
df = df_normalizado

# Mostrar las primeras filas del DataFrame
df.head()

Intentando cargar datos desde: https://docs.google.com/spreadsheets/d/17rtbwMwGMXvoE4sXdGNXXr19z73mdCSjxTApoG5v_6o/export?format=csv
✅ Datos cargados exitosamente: 189 filas y 20 columnas
⚠️ Algunas fechas no pudieron ser convertidas. Verificar que el formato sea día-mes-año (ej: 3-2-2025)
⚠️ Ejemplo de valor problemático: NaT
✅ Se han normalizado 3000 valores NaN a 0 en el DataFrame

Columnas normalizadas:
- fecha: 137 valores reemplazados
- Apellido: 121 valores reemplazados
- DNI: 95 valores reemplazados
- Categoria: 163 valores reemplazados
- EMAIL: 189 valores reemplazados
- Posición : 141 valores reemplazados
- Peso (kg): 144 valores reemplazados
- Talla (cm): 144 valores reemplazados
- IMC: 145 valores reemplazados
- Talla sentado (cm): 144 valores reemplazados
- kg MA: 144 valores reemplazados
- % MA : 144 valores reemplazados
- Z Adiposo: 144 valores reemplazados
- 6 Pliegues: 144 valores reemplazados
- kg MM: 144 valores reemplazados
- % MM : 144 valores reemplazados
- Z MM: 

,fecha,Apellido,DNI,Categoria,EMAIL,Posición,Peso (kg),Talla (cm),IMC,Talla sentado (cm),...,% MA,Z Adiposo,6 Pliegues,kg MM,% MM,Z MM,kg de MO,IMO,Objetivo 1,fecha_str
0,2025-02-03 00:00:00,"Alegre, Diego",30281398,0,0.0,Pilar,0,0,0,0,...,0,0,0,0,0,0,0,0,0,03/02/2025
1,2025-02-03 00:00:00,"Alvarez, Dylan",45742224,0,0.0,Wing,"82,1","177,00","26,20",92,...,"22,78","-1,38",71,"41,58","50,65","2,67","8,62","4,82",Aumento de Masa Muscular,03/02/2025
2,2025-02-03 00:00:00,"Amigorena, Gastón",26106150,0,0.0,Pilar,0,0,0,0,...,0,0,0,0,0,0,0,0,0,03/02/2025
3,2025-02-03 00:00:00,"Amigorena, Valentin",46892621,0,0.0,Wing,0,0,0,0,...,0,0,0,0,0,0,0,0,0,03/02/2025
4,2025-03-11 00:00:00,"Arévalo, Michay Federico",45742426,0,0.0,Medio Scrum,"64,7","164,00","24,05",90,...,"21,8","-1,62",58,"32,6","50,4","2,36","6,5","4,9",Aumento de Masa Muscular,11/03/2025


## Información General del DataFrame

Veamos información básica sobre los datos cargados.

In [26]:
df.columns


Index(['fecha', 'Apellido', 'DNI', 'Categoria', 'EMAIL', 'Posición ',
       'Peso (kg)', 'Talla (cm)', 'IMC', 'Talla sentado (cm)', 'kg MA',
       '% MA ', 'Z Adiposo', '6 Pliegues', 'kg MM', '% MM ', 'Z MM',
       'kg de MO', 'IMO', 'Objetivo 1', 'fecha_str'],
      dtype='object')

In [35]:
def mostrar_info_jugador(apellido):
    """
    Muestra un dashboard con información personal del jugador seleccionado.
    
    Args:
        apellido (str): Apellido del jugador a buscar
    """
    try:
        # Buscar el jugador en el DataFrame (ignorando mayúsculas/minúsculas)
        jugador = df[df['Apellido'].str.contains(apellido, case=False, na=False)]
        
        if len(jugador) == 0:
            print(f"❌ No se encontró ningún jugador con el apellido '{apellido}'")
            return
        
        # Tomar el primer registro si hay múltiples coincidencias
        jugador = jugador.iloc[0]
        
        # Crear el dashboard usando caracteres ASCII
        print("\n" + "="*50)
        print("🏉 DASHBOARD DE COMPOSICIÓN CORPORAL - JUGADOR DE RUGBY")
        print("="*50)
        
        print("\n🧍‍♂️ INFORMACIÓN PERSONAL")
        print("-"*30)
        print(f"👤 Nombre: {jugador['Apellido']}")
        
        # Verificar si existe cada columna antes de mostrarla
        if 'Posición' in jugador:
            print(f"🏈 Posición: {jugador['Posición']}")
        elif 'Posicion' in jugador:  # Alternativa sin tilde
            print(f"🏈 Posición: {jugador['Posicion']}")
        
        if 'Talla (cm)' in jugador:
            print(f"📏 Altura: {jugador['Talla (cm)']} cm")
        elif 'Altura' in jugador:
            print(f"📏 Altura: {jugador['Altura']} cm")
        
        if 'Peso (kg)' in jugador:
            print(f"⚖️ Peso: {jugador['Peso (kg)']} kg")
        elif 'Peso' in jugador:
            print(f"⚖️ Peso: {jugador['Peso']} kg")
        
        if 'fecha_str' in jugador:
            print(f"📅 Última medición: {jugador['fecha_str']}")
        
        if 'Objetivo 1' in jugador:
            print(f"🎯 Objetivo: {jugador['Objetivo 1']}")
        elif 'Objetivo' in jugador:
            print(f"🎯 Objetivo: {jugador['Objetivo']}")
        
        print("\n" + "="*50)
        
    except Exception as e:
        print(f"❌ Error al procesar la información: {e}")
        
        # Mostrar las columnas disponibles para ayudar a depurar
        print("\nColumnas disponibles en el DataFrame:")
        for i, col in enumerate(df.columns):
            print(f"{i+1}. {col}")
        
    return

In [ ]:
d = mostrar_info_jugador("Alvarez, Dylan")  # Cambia "Gonzalez" por el apellido que desees buscar


🏉 DASHBOARD DE COMPOSICIÓN CORPORAL - JUGADOR DE RUGBY

🧍‍♂️ INFORMACIÓN PERSONAL
------------------------------
👤 Nombre: Alvarez, Dylan
📏 Altura: 177,00 cm
⚖️ Peso: 82,1 kg
📅 Última medición: 03/02/2025
🎯 Objetivo: Aumento de Masa Muscular



## Visualización de Datos

A continuación, creamos algunas visualizaciones para entender mejor los datos.

In [ ]:
# Verificar si existen columnas numéricas para graficar
columnas_numericas = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

if 'fecha' in df.columns and len(columnas_numericas) > 0:
    print(f"Columnas numéricas disponibles para graficar: {columnas_numericas}")
    
    # Seleccionar algunas columnas numéricas para graficar (máximo 5)
    columnas_para_graficar = columnas_numericas[:5]
    
    # Crear un gráfico de líneas para las columnas seleccionadas
    plt.figure(figsize=(14, 8))
    for columna in columnas_para_graficar:
        plt.plot(df['fecha'], df[columna], marker='o', label=columna)
    
    plt.title('Evolución de Variables en el Tiempo', fontsize=16)
    plt.xlabel('Fecha', fontsize=12)
    plt.ylabel('Valor', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Graficamos un mapa de calor para ver correlaciones
    if len(columnas_numericas) > 1:
        plt.figure(figsize=(12, 10))
        correlation_matrix = df[columnas_numericas].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
        plt.title('Matriz de Correlación entre Variables', fontsize=16)
        plt.tight_layout()
        plt.show()
else:
    print("No hay suficientes datos numéricos o de fecha para crear visualizaciones")

## Gráficos Interactivos con Plotly

Creamos visualizaciones interactivas para un análisis más detallado.

## Exportación de Datos

Puedes exportar los datos procesados a diferentes formatos si es necesario.